In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd gdrive/

/content/gdrive


In [3]:
cd MyDrive/

/content/gdrive/MyDrive


In [4]:
cd END/s8

/content/gdrive/MyDrive/END/s8


In [5]:
ls

dev_light.json  LICENSE  train_light.json


In [6]:
import json
with open('train_light.json') as f:
  input = json.load(f)

In [7]:
len(input)

10036

In [8]:
input[1].keys()

dict_keys(['annotations', 'id', 'question'])

In [9]:
input[0]["question"]

'When did the simpsons first air on television?'

In [10]:
input

[{'annotations': [{'qaPairs': [{'answer': ['April 19, 1987'],
      'question': 'When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?'},
     {'answer': ['December 17, 1989'],
      'question': 'When did the Simpsons first air as a half-hour prime time show?'}],
    'type': 'multipleQAs'}],
  'id': '-4469503464110108318',
  'question': 'When did the simpsons first air on television?'},
 {'annotations': [{'answer': ['David Morse'], 'type': 'singleAnswer'}],
  'id': '4790842463458965203',
  'question': 'Who played george washington in the john adams series?'},
 {'annotations': [{'qaPairs': [{'answer': ['18 years of age', '18'],
      'question': 'What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?'},
     {'answer': ['19'],
      'question': 'What is the legal age of marriage, without parental consent or other authorization, in Nebraska?'},
     {'answer': ['21'],
      'question': 

In [11]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [14]:
SRC = data.Field( tokenize = 'spacy',init_token = '<sos>', eos_token = '<eos>', lower = True)
TRG = data.Field( tokenize = 'spacy',init_token = '<sos>', eos_token = '<eos>', lower = True)

In [45]:
fields = [('src', SRC),('trg',TRG)]

In [46]:
input[3]['annotations']

[{'qaPairs': [{'answer': ['Elizabeth Ashley'],
    'question': 'Who starred in barefoot in the park on broadway as Corie Bratter?'},
   {'answer': ['Kurt Kasznar'],
    'question': 'Who starred in barefoot in the park on broadway as Victor Velasco?'},
   {'answer': ['Mildred Natwick'],
    'question': 'Who starred in barefoot in the park on broadway as Mrs. Banks?'},
   {'answer': ['Robert Redford'],
    'question': 'Who starred in barefoot in the park on broadway as Paul Bratter?'},
   {'answer': ['Herbert Edelman'],
    'question': 'Who starred in barefoot in the park on broadway as Telephone Man?'},
   {'answer': ['Joseph Keating'],
    'question': 'Who starred in barefoot in the park on broadway as Delivery Man?'}],
  'type': 'multipleQAs'}]

In [47]:
question,answer =[],[]
for i in range(len(input)):
  ano = input[i]['annotations'][0]
  if 'qaPairs' in ano.keys():
    for j in range(len(ano['qaPairs'])):
      question.append(ano['qaPairs'][j]['question'])
      answer.append(ano['qaPairs'][j]['answer'][0])
      #print(ano['qaPairs'][j])
len(answer),len(question)

(13955, 13955)

In [48]:
example = [data.Example.fromlist([question[i],answer[i]], fields) for i in range(len(answer))]

In [49]:
ambignqlight = data.Dataset(example, fields)

In [50]:
train_data, test_data = ambignqlight.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [51]:
len(train_data),len(test_data)

(11862, 2093)

In [52]:
train_data,valid_data = train_data.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [53]:
vars(example[10])

{'src': ['who',
  'starred',
  'in',
  'barefoot',
  'in',
  'the',
  'park',
  'on',
  'broadway',
  'as',
  'telephone',
  'man',
  '?'],
 'trg': ['herbert', 'edelman']}

In [54]:
len(train_data),len(valid_data)

(10083, 1779)

In [55]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, sort=False,
    device = device)

In [71]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [72]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [73]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [74]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [75]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5856, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2983, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=2983, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [76]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,235,879 trainable parameters


In [77]:
optimizer = optim.Adam(model.parameters())

In [78]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [79]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [80]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [81]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [82]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 5.396 | Train PPL: 220.436
	 Val. Loss: 4.215 |  Val. PPL:  67.711
Epoch: 02 | Time: 0m 6s
	Train Loss: 4.577 | Train PPL:  97.202
	 Val. Loss: 4.140 |  Val. PPL:  62.811
Epoch: 03 | Time: 0m 6s
	Train Loss: 4.394 | Train PPL:  80.994
	 Val. Loss: 4.034 |  Val. PPL:  56.514
Epoch: 04 | Time: 0m 6s
	Train Loss: 4.130 | Train PPL:  62.196
	 Val. Loss: 3.770 |  Val. PPL:  43.397
Epoch: 05 | Time: 0m 6s
	Train Loss: 3.941 | Train PPL:  51.473
	 Val. Loss: 3.710 |  Val. PPL:  40.863
Epoch: 06 | Time: 0m 6s
	Train Loss: 3.797 | Train PPL:  44.575
	 Val. Loss: 3.646 |  Val. PPL:  38.322
Epoch: 07 | Time: 0m 6s
	Train Loss: 3.650 | Train PPL:  38.491
	 Val. Loss: 3.589 |  Val. PPL:  36.213
Epoch: 08 | Time: 0m 6s
	Train Loss: 3.520 | Train PPL:  33.776
	 Val. Loss: 3.609 |  Val. PPL:  36.915
Epoch: 09 | Time: 0m 6s
	Train Loss: 3.431 | Train PPL:  30.910
	 Val. Loss: 3.580 |  Val. PPL:  35.864
Epoch: 10 | Time: 0m 6s
	Train Loss: 3.302 | Train PPL:  27.162


In [83]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.640 | Test PPL:  38.075 |
